# Educational neural networks in Python

This code is loosely inspired to [Andrej Karpathy](https://cs.stanford.edu/people/karpathy/)'s excellent but discontinued [Hacker's guide to Neural Networks](http://karpathy.github.io/neuralnets/).

This implementation is not a one-to-one translation of the original javascript code into Python, but [there](https://github.com/urwithajit9/HG_NeuralNetwork) [are](https://github.com/johnashu/hackers_guide_to_neural_networks) [many](https://github.com/saiashirwad/Hackers-Guide-To-Neural-Networks-Python) [repositories](https://github.com/pannous/karpathy_neuralnets_python) [on](https://github.com/techniquark/Hacker-s-Guide-to-Neural-Networks-in-Python) [Github](https://github.com/Mutinix/hacker-nn/) that closely match it line-by-line. Use those to follow along the blog post.

The purpose of this version is to simplify network definition and automate the computation of forward and backward passes. Both these tasks are exploded and manual (for clarity's sake!) in Karpathy's code.

# Base case: single gate in the circuit

Shows a single gate implementing f(x,y) = xy.

In [ ]:
from gates import *

a = Constant(3)
b = Constant(-1)
ab = MulGate(a, b)

print 'a * b = ', ab.compute()

a * b =  -3.0


The code above is equivalent to:

In [18]:
from utils.sugar import *

a = const(3)
b = const(-1)
ab = a * b

print 'a * b = ', ab.compute()


a * b =  -3.0
